In [1]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

apiKey = os.getenv('API_KEY')

In [2]:
demographic = pd.read_csv('Demographic_Data.csv')
geographic = pd.read_csv('Geographic_Data.csv')

In [3]:
demographic

,Location,Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher)
0,Los Angeles,10039107,3276,89,60000,32
1,Paris,2140526,21383,98,45000,29
2,Tokyo,13929286,6169,100,55000,37
3,Antananarivo,1391433,3097,69,1000,10
4,Nairobi,4397073,6000,61,1500,12
5,Lima,9674755,3220,81,4500,15


In [4]:
geographic

,Location,Altitude (m),Proximity to Industry (km)
0,Los Angeles,89,5.0
1,Paris,35,3.0
2,Tokyo,40,2.0
3,Antananarivo,1276,0.5
4,Nairobi,1795,1.0
5,Lima,1540,1.5


In [5]:
location_data = pd.merge(geographic, demographic, on='Location')
location_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher)
0,Los Angeles,89,5.0,10039107,3276,89,60000,32
1,Paris,35,3.0,2140526,21383,98,45000,29
2,Tokyo,40,2.0,13929286,6169,100,55000,37
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10
4,Nairobi,1795,1.0,4397073,6000,61,1500,12
5,Lima,1540,1.5,9674755,3220,81,4500,15


In [6]:
coord = pd.DataFrame(
  {
    'lat': [  34.052235, 48.866667,  34.886306, -18.777192,  0.170945,  -9.181352],
    'lon': [-118.243683,  2.333333, 134.379711,  46.854328, 37.903969, -75.002365],
    'Location': ['Los Angeles', 'Paris', 'Tokyo', 'Antananarivo', 'Nairobi', 'Lima']
  })
coord

,lat,lon,Location
0,34.052235,-118.243683,Los Angeles
1,48.866667,2.333333,Paris
2,34.886306,134.379711,Tokyo
3,-18.777192,46.854328,Antananarivo
4,0.170945,37.903969,Nairobi
5,-9.181352,-75.002365,Lima


In [7]:
location_data = pd.merge(location_data, coord, on='Location')
location_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),lat,lon
0,Los Angeles,89,5.0,10039107,3276,89,60000,32,34.052235,-118.243683
1,Paris,35,3.0,2140526,21383,98,45000,29,48.866667,2.333333
2,Tokyo,40,2.0,13929286,6169,100,55000,37,34.886306,134.379711
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10,-18.777192,46.854328
4,Nairobi,1795,1.0,4397073,6000,61,1500,12,0.170945,37.903969
5,Lima,1540,1.5,9674755,3220,81,4500,15,-9.181352,-75.002365


In [15]:
def get_polution_aqi_data(row, apiKey):
    lat = row['lat']
    lon = row['lon']
    response = requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={apiKey}')
    data = response.json()
    return data['list'][0]['main']['aqi']

def get_polution_components_data(row, apiKey):
    lat = row['lat']
    lon = row['lon']
    response = requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={apiKey}')
    data = response.json()
    return data['list'][0]['components']

location_data['AQI Pollution'] = location_data.apply(lambda row: get_polution_aqi_data(row, apiKey), axis=1)
components_df = location_data.apply(lambda row: pd.Series(get_polution_components_data(row, apiKey)), axis=1)

location_data = location_data.join(components_df)

location_data.drop(columns='lat', axis=1, inplace=True)
location_data.drop(columns='lon', axis=1, inplace=True)
location_data['date'] = datetime.now().date()

location_data

TypeError: dtype '<class 'datetime.datetime'>' not understood